Guideline Level

<center><p style="font-size:50px; font-weight:bold">
Python Basic Introduction
</p></center>

Bioinformatics Algorithms

# Packages

In [ ]:
# Packages and modules
## OS
import re
import os
import shutil
import subprocess
import sys

## String
import string

## Scientific Calculation
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import scipy
import Bio


# Sorting


# Sequence Alignment Algorithms

## Global Sequence Alignment

In [ ]:
# ---------- Needle Wunsch Algorithm ---------- #


## Local Sequence Alignment

**Ab**
- Partial match

In [38]:
# ---------- Smith Waterman (SW) Algorithm ---------- #
#Perform Smith Waterman Alignment in Python (from first principles)

#contains rows lists each of length cols initially set to 0
#index as my_matrix[1][2] my_matrix[R][C]
def create_matrix(rows, cols):
    my_matrix = [[0 for col in range(cols+1)] for row in range(rows+1)]
    return my_matrix

#x is row index, y is column index
# follows[r][c]
def calc_score(matrix, x, y):

  # print("seq1:",sequence1[y- 1]," seq2: "+sequence2[x - 1],"x:",x," y:",y)

   sc = seqmatch if sequence1[y- 1] == sequence2[x - 1] else seqmismatch

   base_score = matrix[x - 1][y - 1] + sc
   insert_score   = matrix[x - 1][y] + seqgap
   delete_score = matrix[x][y - 1] + seqgap
   v=max(0, base_score, insert_score, delete_score)
   return v

#makes a single traceback step
def traceback(mymatrix,maxv):
    x=maxv[0]
    y=maxv[-1]
    val=mymatrix[x][y]

    #oops bug...now fixed
    sc = seqmatch if sequence1[y - 2] == sequence2[x - 2] else seqmismatch
    base_score = mymatrix[x - 1][y - 1] + sc
    if base_score==val:
        return [x-1,y-1]

    insert_score = mymatrix[x - 1][y] + seqgap
    if insert_score==val:
        return [x-1,y]
    else:
        return [x,y - 1]


#builds the initial scoring matrix used for traceback
def build_matrix(mymatrix):

    rows=len(mymatrix)
    cols=len(mymatrix[0])

    for i in range(1, rows):
         for j in range(1, cols):
          mymatrix[i][j]  = calc_score(mymatrix, i, j)

    return mymatrix

#gets the max value from the built matrix
def get_max(mymatrix):

     max=mymatrix[0][0]
     mrow=0
     mcol=0

     rows = len(mymatrix)
     cols = len(mymatrix[0])

     for i in range(1, rows):
         for j in range(1, cols):
             if mymatrix[i][j]>max:
                 max=mymatrix[i][j]
                 mrow=i
                 mcol=j
     print("max score: ",max)
     return [mrow,mcol]

#print out the best scoring path from the SW matrix
def print_matrix(mymatrix):
    rows = len(mymatrix)
    cols = len(mymatrix[0])
    s1="  " +sequence1
    s2=" "+sequence2

    print("Dimensions: r= %2d , c= %2d" % (rows, cols))

    for a in s1:
        print(a, end ="")
        print(" \t", end ="")
    print("\n",end="")

    for i in range(0, rows):
        print(s2[i], end ="")
        print(" \t", end ="")
        for j in range(0, cols):
           print("%02d\t" % (mymatrix[i][j]),end="")
        print("\n",end="")

#print out the traceback of the best scoring alignment
def print_traceback(mymatrix):
    #this will print as expected with internal gaps
   print("Building traceback...")
   maxv=get_max(mymatrix)
   print(maxv)

   #traverse the matrix to find the traceback elements
   #if more than one path just pick one
   topstring=""
   midstring=""
   bottomstring=""

   #pad the sequences so indexes into the sequences match the matrix indexes
   asequence1 = "#" + sequence1
   asequence2 = "#" + sequence2

   # add first element and go to previous
   topstring += asequence1[maxv[-1]]
   bottomstring += asequence2[maxv[0]]
   if asequence1[maxv[-1]] == asequence2[maxv[0]]:
       midstring += "|"
   else:
       midstring += " "

   #here we need to store the old position so we can track if it is an insertion of deletion
   #code assumes highest score cannot be a gap!
   old_maxv=maxv

   #add the rest of the elements
   search=True
   while(search):
       maxv=traceback(mymatrix,maxv)
       if(mymatrix[maxv[0]][maxv[-1]]==0):
           search=False
           continue

        #deal with single gaps or matches
       if(old_maxv[-1]==maxv[-1]):
           topstring+="-"
           bottomstring +=asequence2[maxv[0]]
       elif(old_maxv[0] == maxv[0]):
            # insertion or deletion
           topstring += asequence1[maxv[-1]]
           bottomstring += "-"
       else:
           # add the next element and go to previous
           topstring += asequence1[maxv[-1]]
           bottomstring += asequence2[maxv[0]]

       if (asequence1[maxv[-1] ] == asequence2[maxv[0] ]) & (old_maxv[-1]!=maxv[-1]) & (old_maxv[0] != maxv[0]):
           midstring += "|"
       else:
           midstring += " "
       old_maxv = maxv

   print(topstring[::-1])
   print(midstring[::-1])
   print(bottomstring[::-1])



#build the SW alignment...
def perform_smith_waterman():
#values for weights
  global  seqmatch
  global  seqmismatch
  global  seqgap
  global sequence1
  global sequence2

#note these are not the exact weights used in the original SW paper
  seqmatch =1
  seqmismatch=-1
  seqgap=-1

  #input sequences
  #sequence1="AGTGATAAACTAGTAATTTTT"
  #sequence2="TTGGGGGTAAACAGGGG"


  sequence1 = "GTGTATT"
  sequence2 = "GTGTTATT"

  print("Sequence1: "+sequence1);
  print("Sequence2: "+sequence2);

  mymatrix=create_matrix(len(sequence2), len(sequence1))
  mymatrix=build_matrix(mymatrix)
  print_matrix(mymatrix)

  print_traceback(mymatrix)

##this calls the SW algorithm when the script loads
perform_smith_waterman()


Sequence1: GTGTATT
Sequence2: GTGTTATT
Dimensions: r=  9 , c=  8
  	  	G 	T 	G 	T 	A 	T 	T 	
  	00	00	00	00	00	00	00	00	
G 	00	01	00	01	00	00	00	00	
T 	00	00	02	01	02	01	01	01	
G 	00	01	01	03	02	01	00	00	
T 	00	00	02	02	04	03	02	01	
T 	00	00	01	01	03	03	04	03	
A 	00	00	00	00	02	04	03	03	
T 	00	00	01	00	01	03	05	04	
T 	00	00	01	00	01	02	04	06	
Building traceback...
max score:  6
[8, 7]
GTG-TATT
||| ||||
GTGTTATT


# Affine Gaps

In [39]:
# ---------- Smith Waterman (SW) Algorithm with affine gaps ---------- #
#Perform Smith Waterman Alignment in Python (from first principles)

#contains rows lists each of length cols initially set to 0
#index as my_matrix[1][2] my_matrix[R][C]
def create_matrix(rows, cols, D, E):
    # Original create_matrix section
    def x(i, j):
        if i > 0 and j == 0:
            return MIN
        elif j > 0:
            return D + (E * j)
        else:
            return 0

    def y(i, j):
        if j > 0 and i == 0:
            return MIN
        elif i > 0:
            return D + (E * i)
        else:
            return 0

    def m(i, j):
        if j == 0 and i == 0:
            return 0
        elif j == 0 or i == 0:
            return MIN
        else:
            return 0

    m_matrix = [[m(col, row) for col in range(cols+1)] for row in range(rows+1)]
    x_matrix = [[x(col, row) for col in range(cols+1)] for row in range(rows+1)]
    y_matrix = [[y(col, row) for col in range(cols+1)] for row in range(rows+1)]

    return m_matrix, x_matrix, y_matrix

def match(seq1, seq2, i, j):
    if seq2[i-1] == seq1[j-1]:
        return seqmatch
    else:
        return seqmismatch

#builds the initial scoring matrix used for traceback
def build_matrix(m_matrix, x_matrix, y_matrix, sequence1, sequence2, D, E):

    rows=len(sequence2)
    cols=len(sequence1)

    for j in range(1, cols+1):
        for i in range(1, rows+1):
            x_matrix[i][j] = max((D + E + m_matrix[i][j-1]), (E + x_matrix[i][j-1]), (D + E + y_matrix[i][j-1]))
            y_matrix[i][j] = max((D + E + m_matrix[i-1][j]), (D + E + x_matrix[i-1][j]), (E + y_matrix[i-1][j]))
            m_matrix[i][j] = max(match(sequence1, sequence2, i, j) + m_matrix[i-1][j-1], x_matrix[i][j], y_matrix[i][j])

    return m_matrix, x_matrix, y_matrix


#print out the best scoring path from the SW matrix
def print_matrix(mymatrix):
    rows = len(mymatrix)
    cols = len(mymatrix[0])
    s1=" " +sequence1
    s2=" "+sequence2

    print("Dimensions: r= %2d , c= %2d" % (rows, cols))

    for a in s1:
        print(a, end ="")
        print(" \t", end ="")
    print("\n",end="")

    for i in range(0, rows):
        print(s2[i], end ="")
        print(" \t", end ="")
        for j in range(0, cols):
           print("%02d\t" % (mymatrix[i][j]),end="")
        print("\n",end="")

def print_traceback(seq1, seq2, m_matrix, x_matrix, y_matrix):
   #traverse the matrix to find the traceback elements
   #if more than one path just pick one
    topstring=""
    midstring=""
    bottomstring=""

    i = len(seq2)
    j = len(seq1)
    while (i > 0 or j > 0):
        if (i > 0 and j > 0 and m_matrix[i][j] == m_matrix[i-1][j-1] + match(seq1, seq2, i, j)):
            topstring += seq1[j-1]
            bottomstring += seq2[i-1]
            midstring += "|"
            i -= 1; j -= 1
        elif (i>0 and m_matrix[i][j] == y_matrix[i][j]):
            topstring += '-'
            bottomstring += seq2[i-1]
            midstring += " "
            i -= 1
        elif (j>0 and m_matrix[i][j] == x_matrix[i][j]):
            topstring += seq1[j-1]
            bottomstring += '-'
            midstring += " "
            j -= 1

    topstringr = ''.join([topstring[j] for j in range(-1, -(len(topstring)+1), -1)])
    midstringr = ''.join([midstring[j] for j in range(-1, -(len(midstring)+1), -1)])
    bottomstringr = ''.join([bottomstring[j] for j in range(-1, -(len(bottomstring)+1), -1)])

    print(topstringr)
    print(midstringr)
    print(bottomstringr)

In [40]:
#build the SW alignment...
def perform_smith_waterman(D, E):
    #values for weights
    global seqmatch
    global seqmismatch
    global seqgap
    global sequence1
    global sequence2
    global MIN


    #note these are not the exact weights used in the original SW paper
    seqmatch =1
    seqmismatch=-1
    seqgap=-1
    MIN = max(len(sequence1)*E+D, len(sequence2)*E+D)

    #input sequences
    #sequence1="AGTGATAAACTAGTAATTTTT"
    #sequence2="TTGGGGGTAAACAGGGG"

    sequence1 = "GTGTATT"
    sequence2 = "GTGTTATT"

    print("Sequence1: "+sequence1);
    print("Sequence2: "+sequence2);

    m_matrix, x_matrix, y_matrix=create_matrix(len(sequence2), len(sequence1), D, E)
    m_matrix, x_matrix, y_matrix=build_matrix(m_matrix, x_matrix, y_matrix, sequence1, sequence2, D, E)
    for mymatrix in m_matrix, x_matrix, y_matrix:
        print_matrix(mymatrix)

    print_traceback(sequence1, sequence2, m_matrix, x_matrix, y_matrix)

##this calls the SW algorithm when the script loads
perform_smith_waterman(-10, -0.5)



Sequence1: GTGTATT
Sequence2: GTGTTATT
Dimensions: r=  9 , c=  8
  	G 	T 	G 	T 	A 	T 	T 	
  	00	-13	-13	-13	-13	-13	-13	-13	
G 	-13	01	-9	-10	-10	-11	-11	-12	
T 	-13	-9	02	-8	-9	-9	-10	-10	
G 	-13	-10	-8	03	-7	-8	-8	-9	
T 	-13	-10	-9	-7	04	-6	-7	-7	
T 	-13	-11	-9	-8	-6	03	-5	-6	
A 	-13	-11	-10	-8	-7	-5	02	-6	
T 	-13	-12	-10	-9	-7	-8	-4	03	
T 	-13	-12	-11	-9	-8	-8	-7	-3	
Dimensions: r=  9 , c=  8
  	G 	T 	G 	T 	A 	T 	T 	
  	00	-13	-13	-13	-13	-13	-13	-13	
G 	-10	-11	-9	-10	-10	-11	-11	-12	
T 	-11	-11	-12	-8	-9	-9	-10	-10	
G 	-11	-12	-12	-13	-7	-8	-8	-9	
T 	-12	-12	-13	-13	-14	-6	-7	-7	
T 	-12	-13	-13	-14	-14	-15	-7	-8	
A 	-13	-13	-14	-14	-15	-15	-16	-8	
T 	-13	-14	-14	-15	-15	-16	-16	-15	
T 	-14	-14	-15	-15	-16	-16	-17	-17	
Dimensions: r=  9 , c=  8
  	G 	T 	G 	T 	A 	T 	T 	
  	00	-10	-11	-11	-12	-12	-13	-13	
G 	-13	-11	-11	-12	-12	-13	-13	-14	
T 	-13	-9	-12	-12	-13	-13	-14	-14	
G 	-13	-10	-8	-13	-13	-14	-14	-15	
T 	-13	-10	-9	-7	-14	-14	-15	-15	
T 	-13	-11	-9	-8	-6	-15	-15	-16	
A 	-13	-